In [3]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'https://www.flipkart.com/search?q={0}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={1}'  #{0}:format後的第一個位置、{1}:第二個


def get_urls(url, query, start_page, end_page): #建立可自行輸入"查詢值"和"start~end_page"的url清單
    urls = []
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    return urls

def get_parse(url):
    #headers = {"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Mobile Safari/537.36FKUA/msite/0.0.3/msite/Mobile"}
    #res = requests.get(url, headers=headers)
    res = requests.get(url)
    if res.status_code == requests.codes.ok:
        print("請求成功")
        return BeautifulSoup(res.text, "lxml")
    else:
        return print("請求失敗")
    
def get_goods(soup):
    goods = []
    rows = soup.find("div", attrs={"class":"_1HmYoV _35HD7C", "style":"flex-grow:1;overflow:auto"}).find_all("div", "_3liAhj")
    #rows = soup.select("._3liAhj").selct(".hCUpcT+ .col-12-12 ._3O0U0u div:nth-child(1) ._3liAhj")[0]
    for row in rows:
        try:
            name = row.find("a", attrs={"class":"_2cLu-l"}).text
            #name = row.select(".a-size-medium")[0].get_text()
            #name = row.find("div", class_="rush-component").img["alt"]
        except:
            name = None
        try:
            price = row.find("div", attrs={"class":"_1vC4OE"}).text
            #price = row.select(".a-color-base .a-row")[0].get_text()
        except:
            price = None
        try:
            price_2 = row.find("div", "_3auQ3N").text
            #price_2 = row.select(".a-color-secondary .a-color-base")[0].get_text()
        except:
            price_2 = None
        try:
            discount = row.find("div", "VGWI6T").text
        except:
            discount = None
        try:
            star = row.find("div", "hGSR34").text
            #star = row.select("#anonCarousel4 .aok-inline-block , .sg-col-12-of-28 .aok-align-bottom")[0].get_text()
        except:
            star = None
        try:
            reviews = row.find("span", "_38sUEc").text
            #reviews = row.select(".a-size-small .a-size-base")[0].get_text()
        except:
            reviews = None
            
        good = [name, price, price_2, discount, star, reviews]
        goods.append(good)
    return goods

def scraping_bot(urls):
    all_goods = [["品名", "價格", "價格2", "折扣%", "評價", "評論數"]]  #巢狀清單
    
    page = 1
    for url in urls:
        print("抓取: 第" + str(page) + "頁 網路資料中...")
        page = page + 1
        soup = get_parse(url)
        goods = get_goods(soup)
        all_goods = all_goods + goods
        print("等待3秒鐘...")
        if soup.find("li", class_="a-disabled a-last"):
            break   #已經沒有下一頁
        time.sleep(3)
    return all_goods


def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf_8_sig") as fp:  #utf_8_sig:能讓輸出的csv正確顯示中文(utf_8會有亂碼)
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)


if __name__ == "__main__":
#get_urls
    urls = get_urls(url, 'keyboard', 1, 1)
    print(urls)
#get_resource
    goods = scraping_bot(urls)
    df = pd.DataFrame(goods)     #用dataframe列出
    print(df)
    #for good in goods:                #用list列出
    #    print(good)
    save_to_csv(goods, "t.csv")

,name,star,reviews,price,Original_price,reviews
0,Lenovo Legion K200 Wired USB Gaming Keyboard,4.5,12% off,"₹3,499","₹3,999",12% off
1,Cospex Wireless Bluetooth Multifunction Touchp...,None,None,₹999,None,None
2,HP C2500 Wired Combo keyboard and Mouse,4.3,28% off,₹698,₹977,28% off
3,Dell KM117 Wireless Laptop Keyboard,4.4,8% off,"₹1,281","₹1,399",8% off
4,Amkette Xcite NEO Wired USB Laptop Keyboard,4.2,27% off,₹399,₹549,27% off
5,HP Multimedia Slim Wireless Keyboard & Mouse C...,4.3,35% off,"₹1,299","₹1,999",35% off
6,Lenovo KB MICE_BO Wireless combo 100 Eng Wirel...,4.3,40% off,"₹1,198","₹1,999",40% off
7,Dell KB 216 Wired USB Desktop Keyboard,4.4,11% off,₹574,₹649,11% off
8,Zebronics Zeb-K20 Wired USB Desktop Keyboard,4.1,14% off,₹299,₹349,14% off
9,Flipkart SmartBuy KM-206W Wireless Laptop Keyb...,4.3,60% off,₹599,"₹1,499",60% off
